In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from random import sample
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector

In [4]:
#df1=pd.read_csv(r'C:\Users\kacpr\OneDrive - Akademia Górniczo-Hutnicza im. Stanisława Staszica w Krakowie\Licencjat\Dane\otomoto_offers_eng_23-04-2023.csv',delimiter=';')
df5=pd.read_csv('/content/dane_po_transformacji.csv',delimiter=',')

# Skalowanie cech numerycznych

In [5]:
df5=df5.drop('vehicle_brand',axis=1)
df4_log=df5.copy()
df4_log['price']=np.log(df4_log['price'])

df4_sqrt=df5.copy()
df4_sqrt['price']=np.sqrt(df4_sqrt['price'])

Z racji, że nasze zmienne są w różnej skali to przeskaluje je poprzez standard scaler (x-u)/s

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
numeric_features = ['power', 'mileage', 'car_age','engine_displacement','seller_experience']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['seller_type', 'fuel_type','vehicle_model', 'transmission', 'body_type','number_of_doors','state','leather_upholstery',
       'front_lamps_in_led_technology', 'rear_parking_camera',
       'electrically_adjustable_passenger_seat', 'vat']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


y = df5['price']
X = df5.drop('price', axis=1)
X_preprocessed = preprocessor.fit_transform(X)
y_log = df4_log['price']
X_log = df4_log.drop('price', axis=1)
X_log_preprocessed = preprocessor.fit_transform(X_log)

y_sqrt = df4_sqrt['price']
X_sqrt = df4_sqrt.drop('price', axis=1)
X_sqrt_preprocessed = preprocessor.fit_transform(X_sqrt)


**Podział na testowy i uczący zbiór**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2,
                                                    random_state=0,shuffle=True)

X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log_preprocessed, y_log, test_size=0.2,
                                                    random_state=0,shuffle=True)

X_sqrt_train, X_sqrt_test, y_sqrt_train, y_sqrt_test = train_test_split(X_sqrt_preprocessed, y_sqrt, test_size=0.2,
                                                    random_state=0,shuffle=True)

# Regresja liniowa

In [11]:
from sklearn.linear_model import LinearRegression
regresja = LinearRegression()
regresja.fit(X_train, y_train)

y_pred_Reg=regresja.predict(X_test)
MAPE_Reg=mean_absolute_percentage_error(y_pred_Reg,y_test)
MAE_Reg=mean_absolute_error(y_pred_Reg,y_test)


#### Logarytm regresja

In [12]:
regresja.fit(X_log_train, y_log_train)
y_pred_Reg_log=regresja.predict(X_log_test)
MAPE_Reg_log=mean_absolute_percentage_error(np.exp(y_pred_Reg_log),np.exp(y_log_test))
MAE_Reg_log=mean_absolute_error(np.exp(y_pred_Reg_log),np.exp(y_log_test))

Pierwiastek regresja

In [13]:
regresja.fit(X_sqrt_train, y_sqrt_train)
y_pred_Reg_sqrt=regresja.predict(X_sqrt_test)
MAPE_Reg_sqrt=mean_absolute_percentage_error(np.square(y_pred_Reg_sqrt),np.square(y_sqrt_test))
MAE_Reg_sqrt=mean_absolute_error(np.square(y_pred_Reg_sqrt),np.square(y_sqrt_test))

# Ridge

In [14]:
from sklearn.linear_model import Ridge
param_grid = {
    'alpha': [0.1, 1, 10, 100]
}


clf = GridSearchCV(estimator=Ridge(), param_grid=param_grid,scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)
clf.fit(X_train, y_train)
best_params_Ridge = clf.best_params_
alpha1 = best_params_Ridge.get('alpha', None)

ridge=Ridge(alpha=alpha1)
ridge.fit(X_train, y_train)
y_pred_Ridge=ridge.predict(X_test)
MAPE_Ridge=mean_absolute_percentage_error(y_pred_Ridge,y_test)
MAE_Ridge=mean_absolute_error(y_pred_Ridge,y_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ........................alpha=0.1;, score=-0.566 total time=   0.4s
[CV 2/5] END ........................alpha=0.1;, score=-0.547 total time=   0.5s
[CV 3/5] END ........................alpha=0.1;, score=-0.554 total time=   0.4s
[CV 4/5] END ........................alpha=0.1;, score=-0.562 total time=   0.5s
[CV 5/5] END ........................alpha=0.1;, score=-0.542 total time=   0.4s
[CV 1/5] END ..........................alpha=1;, score=-0.566 total time=   0.4s
[CV 2/5] END ..........................alpha=1;, score=-0.547 total time=   0.5s
[CV 3/5] END ..........................alpha=1;, score=-0.554 total time=   0.9s
[CV 4/5] END ..........................alpha=1;, score=-0.562 total time=   2.3s
[CV 5/5] END ..........................alpha=1;, score=-0.543 total time=   1.6s
[CV 1/5] END .........................alpha=10;, score=-0.570 total time=   1.0s
[CV 2/5] END .........................alpha=10;, 

In [15]:
best_params_Ridge

{'alpha': 0.1}

## Ridge logarytm

In [16]:
param_grid = {
    'alpha': [0.001,0.1, 1, 10, 100]
}

clf = GridSearchCV(estimator=Ridge(), param_grid=param_grid,scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)
clf.fit(X_log_train, y_log_train)
best_params_Ridge_log = clf.best_params_
alpha1 = best_params_Ridge_log.get('alpha', None)

ridge=Ridge(alpha=alpha1)
ridge.fit(X_log_train, y_log_train)
y_pred_Ridge_log=ridge.predict(X_log_test)
MAPE_Ridge_log=mean_absolute_percentage_error(np.exp(y_pred_Ridge_log),np.exp(y_log_test))
MAE_Ridge_log=mean_absolute_error(np.exp(y_pred_Ridge_log),np.exp(y_log_test))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ......................alpha=0.001;, score=-0.013 total time=   0.4s
[CV 2/5] END ......................alpha=0.001;, score=-0.013 total time=   0.7s
[CV 3/5] END ......................alpha=0.001;, score=-0.013 total time=   1.2s
[CV 4/5] END ......................alpha=0.001;, score=-0.013 total time=   1.8s
[CV 5/5] END ......................alpha=0.001;, score=-0.013 total time=   1.0s
[CV 1/5] END ........................alpha=0.1;, score=-0.013 total time=   0.9s
[CV 2/5] END ........................alpha=0.1;, score=-0.013 total time=   1.2s
[CV 3/5] END ........................alpha=0.1;, score=-0.013 total time=   1.3s
[CV 4/5] END ........................alpha=0.1;, score=-0.013 total time=   1.2s
[CV 5/5] END ........................alpha=0.1;, score=-0.013 total time=   1.0s
[CV 1/5] END ..........................alpha=1;, score=-0.013 total time=   0.7s
[CV 2/5] END ..........................alpha=1;, 

In [18]:
best_params_Ridge_log

{'alpha': 1}

Pierwiastek Ridge

In [19]:
param_grid = {
    'alpha': [0.001, 0.1, 1, 10, 100]
}

clf = GridSearchCV(estimator=Ridge(), param_grid=param_grid, scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)
clf.fit(X_sqrt_train, y_sqrt_train)
best_params_Ridge_sqrt = clf.best_params_
alpha1 = best_params_Ridge_sqrt.get('alpha', None)

ridge = Ridge(alpha=alpha1)
ridge.fit(X_sqrt_train, y_sqrt_train)
y_pred_Ridge_sqrt = ridge.predict(X_sqrt_test)
MAPE_Ridge_sqrt = mean_absolute_percentage_error(np.square(y_pred_Ridge_sqrt), np.square(y_sqrt_test))
MAE_Ridge_sqrt = mean_absolute_error(np.square(y_pred_Ridge_sqrt), np.square(y_sqrt_test))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ......................alpha=0.001;, score=-0.097 total time=   0.4s
[CV 2/5] END ......................alpha=0.001;, score=-0.095 total time=   0.5s
[CV 3/5] END ......................alpha=0.001;, score=-0.095 total time=   0.5s
[CV 4/5] END ......................alpha=0.001;, score=-0.096 total time=   0.5s
[CV 5/5] END ......................alpha=0.001;, score=-0.096 total time=   0.4s
[CV 1/5] END ........................alpha=0.1;, score=-0.097 total time=   0.4s
[CV 2/5] END ........................alpha=0.1;, score=-0.095 total time=   0.5s
[CV 3/5] END ........................alpha=0.1;, score=-0.095 total time=   0.8s
[CV 4/5] END ........................alpha=0.1;, score=-0.096 total time=   1.0s
[CV 5/5] END ........................alpha=0.1;, score=-0.096 total time=   1.1s
[CV 1/5] END ..........................alpha=1;, score=-0.097 total time=   0.8s
[CV 2/5] END ..........................alpha=1;, 

In [20]:
best_params_Ridge_sqrt

{'alpha': 0.001}

# Lasy losowe

In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
       'n_estimators': [300],
    'max_features': [ 'sqrt', 'log2',15,30,50,],
}



clf = GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_train, y_train)

best_params_Forest = clf.best_params_
max_features1 = best_params_Forest.get('max_features', None)
min_samples_split1 = best_params_Forest.get('min_samples_split', None)
n_estimators1 = best_params_Forest.get('n_estimators', None)
Forest=RandomForestRegressor(n_jobs=-1,max_features=max_features1,n_estimators=n_estimators1)
Forest.fit(X_train, y_train)
y_pred_Las=Forest.predict(X_test)
MAE_las=mean_absolute_error(y_test,y_pred_Las)
MAPE_las=mean_absolute_percentage_error(y_test,y_pred_Las)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END max_features=sqrt, n_estimators=300;, score=-0.153 total time=18.2min
[CV 2/5] END max_features=sqrt, n_estimators=300;, score=-0.146 total time=17.9min
[CV 3/5] END max_features=sqrt, n_estimators=300;, score=-0.149 total time=18.0min
[CV 4/5] END max_features=sqrt, n_estimators=300;, score=-0.157 total time=17.9min
[CV 5/5] END max_features=sqrt, n_estimators=300;, score=-0.152 total time=17.6min
[CV 1/5] END max_features=log2, n_estimators=300;, score=-0.170 total time=18.2min
[CV 2/5] END max_features=log2, n_estimators=300;, score=-0.162 total time=18.2min
[CV 3/5] END max_features=log2, n_estimators=300;, score=-0.167 total time=18.2min
[CV 4/5] END max_features=log2, n_estimators=300;, score=-0.175 total time=18.2min
[CV 5/5] END max_features=log2, n_estimators=300;, score=-0.168 total time=18.2min
[CV 1/5] END max_features=15, n_estimators=300;, score=-0.156 total time=17.5min
[CV 2/5] END max_features=15,

In [ ]:
best_params_Forest

{'max_features': 50, 'n_estimators': 300}

In [ ]:
MAPE_las

0.12167865454798071

## Z logarytmem

In [ ]:
from sklearn.ensemble import RandomForestRegressor
param_grid = {
       'n_estimators': [300],
    'max_features': [ 15,30,50,'sqrt', 'log2'],
}


clf = GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_log_train, y_log_train)

best_params_Forest_log = clf.best_params_
max_features1 = best_params_Forest_log.get('max_features', None)
min_samples_split1 = best_params_Forest_log.get('min_samples_split', None)
min_samples_leaf1 = best_params_Forest_log.get('min_samples_leaf', None)
n_estimators1 = best_params_Forest_log.get('n_estimators', None)
Forest=RandomForestRegressor(n_jobs=-1,max_features=max_features1,n_estimators=n_estimators1)
Forest.fit(X_log_train, y_log_train)
y_pred_Las_log=Forest.predict(X_log_test)
MAPE_las_log=mean_absolute_percentage_error(np.exp(y_log_test),np.exp(y_pred_Las_log))
MAE_las_log=mean_absolute_error(np.exp(y_log_test),np.exp(y_pred_Las_log))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END max_features=15, n_estimators=300;, score=-0.012 total time=17.7min
[CV 2/5] END max_features=15, n_estimators=300;, score=-0.012 total time=17.5min
[CV 3/5] END max_features=15, n_estimators=300;, score=-0.012 total time=17.7min
[CV 4/5] END max_features=15, n_estimators=300;, score=-0.012 total time=17.7min
[CV 5/5] END max_features=15, n_estimators=300;, score=-0.012 total time=17.7min
[CV 1/5] END max_features=30, n_estimators=300;, score=-0.011 total time=17.7min
[CV 2/5] END max_features=30, n_estimators=300;, score=-0.011 total time=17.7min
[CV 3/5] END max_features=30, n_estimators=300;, score=-0.011 total time=17.5min
[CV 4/5] END max_features=30, n_estimators=300;, score=-0.011 total time=17.8min
[CV 5/5] END max_features=30, n_estimators=300;, score=-0.011 total time=17.6min
[CV 1/5] END max_features=50, n_estimators=300;, score=-0.011 total time=19.2min
[CV 2/5] END max_features=50, n_estimators=300;, 

In [ ]:
best_params_Forest_log

{'max_features': 50, 'n_estimators': 300}

In [ ]:
MAPE_las_log

0.11426530497356895

### Z pierwiastkiem

In [ ]:
from sklearn.ensemble import RandomForestRegressor
param_grid = {
       'n_estimators': [300],
    'max_features': [ 50,'sqrt', 'log2'],
}


clf = GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_sqrt_train, y_sqrt_train)

best_params_Forest_sqrt = clf.best_params_
max_features1 = best_params_Forest_sqrt.get('max_features', None)
min_samples_split1 = best_params_Forest_sqrt.get('min_samples_split', None)
min_samples_leaf1 = best_params_Forest_sqrt.get('min_samples_leaf', None)
n_estimators1 = best_params_Forest_sqrt.get('n_estimators', None)

Forest = RandomForestRegressor(
    n_jobs=-1,
    max_features=max_features1,
    n_estimators=n_estimators1
)
Forest.fit(X_sqrt_train, y_sqrt_train)
y_pred_Las_sqrt = Forest.predict(X_sqrt_test)
MAPE_las_sqrt = mean_absolute_percentage_error(np.square(y_sqrt_test), np.square(y_pred_Las_sqrt))
MAE_las_sqrt = mean_absolute_error(np.square(y_sqrt_test), np.square(y_pred_Las_sqrt))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END max_features=50, n_estimators=300;, score=-0.059 total time=19.2min
[CV 2/5] END max_features=50, n_estimators=300;, score=-0.057 total time=19.0min
[CV 3/5] END max_features=50, n_estimators=300;, score=-0.058 total time=19.1min
[CV 4/5] END max_features=50, n_estimators=300;, score=-0.059 total time=19.1min
[CV 5/5] END max_features=50, n_estimators=300;, score=-0.059 total time=19.1min
[CV 1/5] END max_features=sqrt, n_estimators=300;, score=-0.067 total time=17.3min
[CV 2/5] END max_features=sqrt, n_estimators=300;, score=-0.064 total time=17.3min
[CV 3/5] END max_features=sqrt, n_estimators=300;, score=-0.066 total time=17.3min
[CV 4/5] END max_features=sqrt, n_estimators=300;, score=-0.067 total time=17.3min
[CV 5/5] END max_features=sqrt, n_estimators=300;, score=-0.066 total time=17.3min
[CV 1/5] END max_features=log2, n_estimators=300;, score=-0.072 total time=17.7min
[CV 2/5] END max_features=log2, n_est

In [ ]:
best_params_Forest_sqrt

{'max_features': 50, 'n_estimators': 300}

In [ ]:
MAPE_las_sqrt

0.11676813163394181

# MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'hidden_layer_sizes': [
        (500), (300),
        (500, 500),
        (500, 300),
        (500, 500, 500),
        (500, 300, 150),
    ],
    'activation': ['relu'],
    'learning_rate': ['constant'],
    'batch_size': ['auto'],
    'early_stopping':[True]
}


clf = GridSearchCV(estimator=MLPRegressor(), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_train, y_train)

best_params_MLP = clf.best_params_
hidden_layer_sizes1 = best_params_MLP.get('hidden_layer_sizes', None)
activation1 = best_params_MLP.get('activation', None)
learning_rate1 = best_params_MLP.get('learning_rate', None)
batch_size1 = best_params_MLP.get('batch_size', None)
Network=MLPRegressor(hidden_layer_sizes=hidden_layer_sizes1,activation=activation1,learning_rate=learning_rate1,batch_size=batch_size1,early_stopping=True)
Network.fit(X_train, y_train)
y_pred_MLP=Network.predict(X_test)
MAE_MLP=mean_absolute_error(y_test,y_pred_MLP)
MAPE_MLP=mean_absolute_percentage_error(y_test,y_pred_MLP)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.161 total time=12.0min


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.158 total time=11.8min


...

[CV 3/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=300, learning_rate=constant;, score=-0.192 total time= 7.6min


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 4/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=300, learning_rate=constant;, score=-0.186 total time= 7.7min


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 5/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=300, learning_rate=constant;, score=-0.187 total time= 8.3min
[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=(500, 500), learning_rate=constant;, score=-0.122 total time=12.4min
[CV 2/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=(500, 500), learning_rate=constant;, score=-0.112 total time=21.7min
[CV 3/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=(500, 500), learning_rate=constant;, score=-0.120 total time=10.7min
[CV 4/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=(500, 500), learning_rate=constant;, score=-0.119 total time=31.2min
[CV 5/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=(500, 500), learning_rate=constant;, score=-0.119 total time=18.6min
[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hid

In [ ]:
MAPE_MLP

0.10568736025841617

In [ ]:
best_params_MLP

{'activation': 'relu',
 'batch_size': 'auto',
 'early_stopping': True,
 'hidden_layer_sizes': (500, 300, 150),
 'learning_rate': 'constant'}

### z logarytmem

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'hidden_layer_sizes': [
        (500), (300),
        (500, 500),
        (500, 300),
        (500, 500, 500),
        (500, 300, 150),
    ],
    'activation': ['relu'],
    'learning_rate': ['constant'],
    'batch_size': ['auto'],
    'early_stopping':[True]
}



clf = GridSearchCV(estimator=MLPRegressor(), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_log_train, y_log_train)

best_params_MLP_log = clf.best_params_
hidden_layer_sizes1 = best_params_MLP_log.get('hidden_layer_sizes', None)
activation1 = best_params_MLP_log.get('activation', None)
learning_rate1 = best_params_MLP_log.get('learning_rate', None)
batch_size1 = best_params_MLP_log.get('batch_size', None)
Network=MLPRegressor(hidden_layer_sizes=hidden_layer_sizes1,activation=activation1,learning_rate=learning_rate1,batch_size=batch_size1)
Network.fit(X_log_train, y_log_train)
y_pred_MLP_log=Network.predict(X_log_test)
MAE_MLP_log=mean_absolute_error(np.exp(y_log_test),np.exp(y_pred_MLP_log))
MAPE_MLP_log=mean_absolute_percentage_error(np.exp(y_log_test),np.exp(y_pred_MLP_log))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.010 total time= 1.3min
[CV 2/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.010 total time= 2.4min
[CV 3/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.010 total time= 1.7min
[CV 4/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.010 total time= 1.6min
[CV 5/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.010 total time= 2.0min
[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=300, learning_rate=constant;, score=-0.010 total time= 1.3min
[CV 2/5] END activation=relu, batch_size=auto,

In [ ]:
MAPE_MLP_log

0.10976407264764004

In [ ]:
best_params_MLP_log

{'activation': 'relu',
 'batch_size': 'auto',
 'early_stopping': True,
 'hidden_layer_sizes': 300,
 'learning_rate': 'constant'}

## Z pierwiastkiem

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
param_grid = {
    'hidden_layer_sizes': [
        (500), (300),
        (500, 500),
        (500, 300),
        (500, 500, 500),
        (500, 300, 150),
    ],
    'activation': ['relu'],
    'learning_rate': ['constant'],
    'batch_size': ['auto'],
    'early_stopping':[True]
}
clf = GridSearchCV(estimator=MLPRegressor(), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_sqrt_train, y_sqrt_train)

best_params_MLP_sqrt = clf.best_params_
hidden_layer_sizes1 = best_params_MLP_sqrt.get('hidden_layer_sizes', None)
activation1 = best_params_MLP_sqrt.get('activation', None)
learning_rate1 = best_params_MLP_sqrt.get('learning_rate', None)
batch_size1 = best_params_MLP_sqrt.get('batch_size', None)
Network = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes1, activation=activation1, learning_rate=learning_rate1, batch_size=batch_size1)
Network.fit(X_sqrt_train, y_sqrt_train)
y_pred_MLP_sqrt = Network.predict(X_sqrt_test)
MAE_MLP_sqrt = mean_absolute_error(np.square(y_sqrt_test), np.square(y_pred_MLP_sqrt))
MAPE_MLP_sqrt = mean_absolute_percentage_error(np.square(y_sqrt_test), np.square(y_pred_MLP_sqrt))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.052 total time= 3.7min
[CV 2/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.051 total time= 3.4min
[CV 3/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.051 total time= 3.5min
[CV 4/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.052 total time= 3.2min
[CV 5/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=500, learning_rate=constant;, score=-0.051 total time= 3.2min
[CV 1/5] END activation=relu, batch_size=auto, early_stopping=True, hidden_layer_sizes=300, learning_rate=constant;, score=-0.052 total time= 2.4min
[CV 2/5] END activation=relu, batch_size=auto,

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
MAPE_MLP_sqrt

0.10311652662451108

In [ ]:
best_params_MLP_sqrt

{'activation': 'relu',
 'batch_size': 'auto',
 'early_stopping': True,
 'hidden_layer_sizes': 500,
 'learning_rate': 'constant'}

# XGBOOST

In [ ]:
import xgboost as xgb
param_grid = {
    'n_estimators': [300, 1000],
    'max_depth': [1,3,6, 8, 10, 12],
    'learning_rate': [0.0001,0.01, 0.1],
}



clf = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror'), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_train, y_train)

best_params_XGB = clf.best_params_

n_estimators1 = best_params_XGB.get('n_estimators', None)
learning_rate1 = best_params_XGB.get('learning_rate', None)
max_depth1 = best_params_XGB.get('max_depth', None)


xgb_model=xgb.XGBRegressor(objective='reg:linear',n_estimators=n_estimators1,learning_rate=learning_rate1,max_depth=max_depth1)
xgb_model.fit(X_train, y_train)
y_pred_XGB=xgb_model.predict(X_test)
MAE_XGB=mean_absolute_error(y_test,y_pred_XGB)
MAPE_XGB=mean_absolute_percentage_error(y_test,y_pred_XGB)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-1.727 total time=  12.3s
[CV 2/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-1.704 total time=   9.8s
[CV 3/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-1.702 total time=  12.4s
[CV 4/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-1.718 total time=   9.7s
[CV 5/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-1.694 total time=   0.9s
[CV 1/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-1.685 total time=   2.5s
[CV 2/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-1.660 total time=   2.5s
[CV 3/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-1.660 total time=   4.9s
[CV 4/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-1.675 total time=   2.5s
[CV 5/5] END learning_rate=0.0001, max_dept

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:46:05] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [ ]:
best_params_XGB

{'learning_rate': 0.1, 'max_depth': 12, 'n_estimators': 1000}

In [ ]:
MAPE_XGB

0.10192056745290756

### Z logarytmem

In [ ]:
param_grid = {
    'n_estimators': [300, 1000],
    'max_depth': [1,3,6, 8, 10, 12],
    'learning_rate': [0.0001,0.01, 0.1],
}


clf = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror'), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_log_train, y_log_train)

best_params_XGB_log = clf.best_params_
n_estimators1 = best_params_XGB_log.get('n_estimators', None)
learning_rate1 = best_params_XGB_log.get('learning_rate', None)
max_depth1 = best_params_XGB_log.get('max_depth', None)
xgb_model=xgb.XGBRegressor(objective='reg:linear',n_estimators=n_estimators1,learning_rate=learning_rate1,max_depth=max_depth1)
xgb_model.fit(X_log_train, y_log_train)
y_pred_XGB_log=xgb_model.predict(X_log_test)
MAPE_XGB_log=mean_absolute_percentage_error(np.exp(y_log_test),np.exp(y_pred_XGB_log))
MAE_XGB_log=mean_absolute_error(np.exp(y_log_test),np.exp(y_pred_XGB_log))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.070 total time=   3.4s
[CV 2/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.070 total time=   1.0s
[CV 3/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.070 total time=   1.0s
[CV 4/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.071 total time=   1.0s
[CV 5/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.070 total time=   1.0s
[CV 1/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.067 total time=   2.6s
[CV 2/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.067 total time=   2.6s
[CV 3/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.067 total time=   5.0s
[CV 4/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.068 total time=   2.7s
[CV 5/5] END learning_rate=0.0001, max_dept

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:41:30] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [ ]:
best_params_XGB_log

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 1000}

In [ ]:
MAPE_XGB_log

0.099326953291893

## Z pierwiastkiem

In [ ]:
param_grid = {
    'n_estimators': [300, 1000],
    'max_depth': [1,3,6, 8, 10, 12],
    'learning_rate': [0.0001,0.01, 0.1],
}



clf = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror'), param_grid=param_grid,
                   scoring='neg_mean_absolute_percentage_error', verbose=3, cv=5)

clf.fit(X_sqrt_train, y_sqrt_train)

best_params_XGB_sqrt = clf.best_params_
n_estimators1 = best_params_XGB_sqrt.get('n_estimators', None)
learning_rate1 = best_params_XGB_sqrt.get('learning_rate', None)
max_depth1 = best_params_XGB_sqrt.get('max_depth', None)
xgb_model = xgb.XGBRegressor(objective='reg:linear', n_estimators=n_estimators1, learning_rate=learning_rate1, max_depth=max_depth1)
xgb_model.fit(X_sqrt_train, y_sqrt_train)
y_pred_XGB_sqrt = xgb_model.predict(X_sqrt_test)
MAPE_XGB_sqrt = mean_absolute_percentage_error(np.square(y_sqrt_test), np.square(y_pred_XGB_sqrt))
MAE_XGB_sqrt = mean_absolute_error(np.square(y_sqrt_test), np.square(y_pred_XGB_sqrt))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.473 total time=   1.0s
[CV 2/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.473 total time=   0.9s
[CV 3/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.470 total time=   0.9s
[CV 4/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.473 total time=   0.9s
[CV 5/5] END learning_rate=0.0001, max_depth=1, n_estimators=300;, score=-0.465 total time=   3.2s
[CV 1/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.455 total time=   2.6s
[CV 2/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.454 total time=   2.6s
[CV 3/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.453 total time=   2.6s
[CV 4/5] END learning_rate=0.0001, max_depth=1, n_estimators=1000;, score=-0.454 total time=   5.0s
[CV 5/5] END learning_rate=0.0001, max_dept

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:48:23] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [ ]:
best_params_XGB_sqrt

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 1000}

In [ ]:
MAPE_XGB_sqrt

0.1001860573887825